In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gps_map = ccal.load_gps_map(PATH["gps_map.pickle.gz"])

In [ ]:
feature_data = {}

if SETTING["feature_data"] is not None:

    for data_name, data_dict in SETTING["feature_data"].items():

        if "indices" not in data_dict:

            continue

        print(data_name)

        feature_data[data_name] = {
            "df": pd.read_csv(data_dict["file_path"], sep="\t", index_col=0),
            **data_dict,
        }

In [ ]:
sample_data = {}

if SETTING["sample_data"] is not None:

    for data_name, data_dict in SETTING["sample_data"].items():

        if "indices" not in data_dict:

            continue

        print(data_name)

        sample_data[data_name] = {
            "df": pd.read_csv(data_dict["file_path"], sep="\t", index_col=0),
            **data_dict,
        }

In [ ]:
for w_or_h, data_dicts in (("w", feature_data), ("h", sample_data)):

    if SETTING["{}_summary".format(w_or_h)] is None:

        continue

    for factor_name, data_name_indices in SETTING["{}_summary".format(w_or_h)].items():

        for data_name, indices in data_name_indices.items():

            if data_name.startswith("_"):

                continue

            if data_dicts[data_name]["indices"] != "all":

                data_dicts[data_name]["indices"] += indices

In [ ]:
for element_type, w_or_h, elements, element_labels, data_dicts in (
    ("feature", "w", gps_map.w_elements, gps_map.w_element_labels, feature_data),
    ("sample", "h", gps_map.h_elements, gps_map.h_element_labels, sample_data),
):

    for data_name, data_dict in data_dicts.items():

        df = data_dict["df"]

        if data_dict["indices"] == "all":

            indices = df.index

        else:

            indices = data_dict["indices"]

        for index, element_value in df.loc[indices, df.columns & elements].iterrows():

            print(index)

            if element_value.isna().all():

                continue

            if data_dict["type"] == "categorical":

                if element_labels is None:

                    colorscale = ccal.make_colorscale(
                        colors=ccal.COLOR_CATEGORICAL[
                            : element_value.dropna().unique().size
                        ],
                        plot=False,
                    )

                else:

                    colorscale = ccal.make_colorscale(
                        n_category=element_value.dropna().unique().size, plot=False
                    )

            else:

                colorscale = None

            title = "{} GPS Map<br>{}: {}".format(
                SETTING["{}_alias".format(element_type)], data_name, index
            )

            file_name = ccal.make_file_name_from_str(
                "{}_{}.html".format(data_name, index)
            )

            if element_labels is None:

                gps_map_directory_path = PATH["{}|gps_map/".format(w_or_h)]

            else:

                gps_map_directory_path = PATH["{}|hcc|gps_map/".format(w_or_h)]

            html_file_path = os.path.join(gps_map_directory_path, file_name)

            gps_map.plot_gps_map(
                w_or_h,
                annotation_x_element=element_value.to_frame().T,
                annotation_std_maxs=(SETTING["plot_std"],),
                annotation_types=(data_dict["type"],),
                annotation_colorscale=colorscale,
                element_marker_size=SETTING[
                    "gps_map_{}_element_marker_size".format(w_or_h)
                ],
                title=title,
                html_file_path=html_file_path,
            )

            if element_labels is None:

                continue

            label_element_value = element_value.groupby(by=element_labels)

            names = tuple(
                "{} Cluster {}".format(SETTING["{}_alias".format(element_type)], label)
                for label in label_element_value.groups.keys()
            )

            html_file_path = os.path.join(
                PATH["{}|hcc|comparison/".format(w_or_h)], file_name
            )

            xaxis_title = "{} Cluster".format(SETTING["{}_alias".format(element_type)])

            if data_dict["type"] in ("continuous", "categorical"):

                ccal.plot_violin_or_box(
                    (element_value_ for label, element_value_ in label_element_value),
                    names=names,
                    violin_or_box="box",
                    title=title,
                    xaxis_title=xaxis_title,
                    yaxis_title="Value",
                    html_file_path=html_file_path,
                )

            elif data_dict["type"] == "binary":

                ccal.plot_bar(
                    ((name,) for name in names),
                    (
                        (label_density,)
                        for label_density in label_element_value.sum()
                        / label_element_value.size()
                    ),
                    names=names,
                    title=title,
                    xaxis_title=xaxis_title,
                    yaxis_title="Density",
                    html_file_path=html_file_path,
                )